In [1]:
# import numpy as np
# import torch


# SLICE = 1000

# eeg_data = np.load('./run_experiments_ssl_signal/eeg_data.npy', mmap_mode='r')
# emg_data = np.load('./run_experiments_ssl_signal/emg_data.npy', mmap_mode='r')
# subject = np.load('./run_experiments_ssl_signal/subject.npy')
# label = np.load('./run_experiments_ssl_signal/label.npy')
# new_label = np.load('./run_experiments_ssl_signal/new_label.npy')

# idx = np.random.choice(len(eeg_data), SLICE)
# eeg_data = eeg_data[idx]
# emg_data = emg_data[idx]
# subject = subject[idx]
# label = label[idx]
# new_label = new_label[idx]

# np.save('./run_experiments_ssl_signal/eeg_data_sample.npy', eeg_data)
# np.save('./run_experiments_ssl_signal/emg_data_sample.npy', emg_data)
# np.save('./run_experiments_ssl_signal/subject_sample.npy', subject)
# np.save('./run_experiments_ssl_signal/label_sample.npy', label)
# np.save('./run_experiments_ssl_signal/new_label_sample.npy', new_label)

In [2]:
import numpy as np
import torch

subject = np.load('./run_experiments_ssl_signal/subject_sample.npy')
label = np.load('./run_experiments_ssl_signal/label_sample.npy')
new_label = np.load('./run_experiments_ssl_signal/new_label_sample.npy')
eeg_data = np.load('./run_experiments_ssl_signal/eeg_data_sample.npy')
emg_data = np.load('./run_experiments_ssl_signal/emg_data_sample.npy')

subject = torch.from_numpy(subject).long()
label = torch.from_numpy(label).long()
new_label = torch.from_numpy(new_label).long()
eeg_data = torch.from_numpy(eeg_data).float()
emg_data = torch.from_numpy(emg_data).float()

SAMPLE_X = eeg_data
SAMPLE_Y = emg_data
label_tensor = new_label

print(SAMPLE_X[:1].shape)
print(SAMPLE_Y[:1].shape)
print(label_tensor[:1].shape)
# Generate Data Cross Subject

train_idx = [np.where(subject == k)[0] for k in range(0, 20)]
test_idx = [np.where(subject == k)[0] for k in range(20, 25)]

train_idx = np.concatenate(train_idx)
test_idx = np.concatenate(test_idx)

train_major_label, test_major_label = new_label[train_idx], new_label[test_idx]
train_subtle_label, test_subtle_label = label[train_idx], label[test_idx]
train_subj_label, test_subj_label = subject[train_idx], subject[test_idx]

train_X, test_X = SAMPLE_X[train_idx], SAMPLE_X[test_idx]
train_Y, test_Y = SAMPLE_Y[train_idx], SAMPLE_Y[test_idx]

torch.Size([1, 60, 4000])
torch.Size([1, 7, 4000])
torch.Size([1])


In [3]:
import torch
import torch.nn as nn

import torch
import torch.nn as nn

from torch.utils.data import Subset, Dataset
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image

import numpy as np
import random

import numpy as np
import matplotlib.pyplot as plt

import csv
from sklearn import svm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from sklearn.linear_model import LinearRegression

import sklearn
from sklearn import datasets

from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt

class ComplexClassifier(nn.Module):
    def __init__(self, dim_features=128, num_classes = 10):
        super(ComplexClassifier, self).__init__()
        self.fc1 = nn.Linear(dim_features, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc4 = nn.Linear(256, num_classes)  # CIFAR-10 has 10 classes

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)  # No activation, CrossEntropyLoss includes softmax
        return x
    
class NETWORK_F_MLP(nn.Module):
    def __init__(self, input_dim = 784, HIDDEN = 200, out_dim = 200, how_many_layers = 2):
        super(NETWORK_F_MLP, self).__init__()
        self.dim = out_dim
        self.many_layer = how_many_layers
        
        self.fc_list = []
        self.bn_list = []
        
#         self.fc_list.append(nn.Linear(input_dim+20, HIDDEN, bias=True))
        self.fc_list.append(nn.Linear(input_dim, HIDDEN, bias=True))
        self.bn_list.append(nn.BatchNorm1d(HIDDEN))

        for i in range(0, self.many_layer-1):
            self.fc_list.append(nn.Linear(HIDDEN, HIDDEN, bias=True))
            self.bn_list.append(nn.BatchNorm1d(HIDDEN))
            
        self.fc_list = nn.ModuleList(self.fc_list)
        self.bn_list = nn.ModuleList(self.bn_list)

        self.fc_final = nn.Linear(HIDDEN, out_dim, bias=True)

    def forward(self, x):
        
        x = x.reshape(x.shape[0], -1)
        
        for i in range(0, self.many_layer):
            x = self.fc_list[i](x)
            x = torch.relu(x)
            x = self.bn_list[i](x)
        
        x = self.fc_final(x)
        x = torch.sigmoid(x)
        return x

class Advanced1DCNN_channel(nn.Module):
    def __init__(self, input_channel=1, num_classes=100, input_size=4000):
        super(Advanced1DCNN_channel, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(input_channel, 32, kernel_size=5, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=5, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=5, padding=2),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        )
        
        self.conv5 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=5, padding=2),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(int(int(int(int(int(input_size/4)/4)/4)/4))*256, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
        )

        self.fc2 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.fc3 = nn.Linear(512, num_classes)
        
        self.MLP = NETWORK_F_MLP(input_dim = 128, HIDDEN = 2000, out_dim = num_classes, how_many_layers = 2)
        
    def forward(self, x):
        bs, channel = x.shape[0], x.shape[1]
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = out.view(out.size(0), -1)

        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
#         out = torch.sigmoid(out)
        out = self.MLP(out)
        return out

def create_folder_if_not_exists(folder_path):
    # Check if the folder exists
    if not os.path.exists(folder_path):
        # If the folder does not exist, create it
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")
    else:
        print(f"Folder '{folder_path}' already exists.")


# net = Advanced1DCNN_channel(60, 128, 1000)

In [ ]:
import os 

class NETWORK_F_MLP(nn.Module):
    def __init__(self, input_dim = 784, HIDDEN = 200, out_dim = 200, how_many_layers = 2):
        super(NETWORK_F_MLP, self).__init__()
        self.dim = out_dim
        self.many_layer = how_many_layers
        
        self.fc_list = []
        self.bn_list = []
        
#         self.fc_list.append(nn.Linear(input_dim+20, HIDDEN, bias=True))
        self.fc_list.append(nn.Linear(input_dim, HIDDEN, bias=True))
        self.bn_list.append(nn.BatchNorm1d(HIDDEN))

        for i in range(0, self.many_layer-1):
            self.fc_list.append(nn.Linear(HIDDEN, HIDDEN, bias=True))
            self.bn_list.append(nn.BatchNorm1d(HIDDEN))
            
        self.fc_list = nn.ModuleList(self.fc_list)
        self.bn_list = nn.ModuleList(self.bn_list)

        self.fc_final = nn.Linear(HIDDEN, out_dim, bias=True)

    def forward(self, x):
        
        x = x.reshape(x.shape[0], -1)
        
        for i in range(0, self.many_layer):
            x = self.fc_list[i](x)
            x = torch.relu(x)
            x = self.bn_list[i](x)
        
        x = self.fc_final(x)
        x = torch.sigmoid(x)
        return x

class Advanced1DCNN_channel(nn.Module):
    def __init__(self, input_channel=1, num_classes=100, input_size=4000, num_channel=60):
        super(Advanced1DCNN_channel, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=11, padding=5),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=11, padding=5),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=11, padding=5),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=11, padding=5),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(256 * 15, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
        )

        self.fc2 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.fc3 = nn.Linear(512, num_classes)
        
        self.MLP = NETWORK_F_MLP(input_dim = 128*input_channel, HIDDEN = 4000, out_dim = num_classes, how_many_layers = 1)
        
    def forward(self, x):
        bs, channel = x.shape[0], x.shape[1]
        x = x.unsqueeze(2)
        x = x.flatten(0, 1)
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
#         print(out.shape)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = torch.sigmoid(out)
        out = out.reshape(bs, channel, -1)
        out = out.flatten(-2, -1)
        out = self.MLP(out)
        return out

class ComplexClassifier(nn.Module):
    def __init__(self, dim_features=128, num_classes = 10):
        super(ComplexClassifier, self).__init__()
#         self.fc1 = nn.Linear(dim_features, 256)
#         self.bn1 = nn.BatchNorm1d(256)
#         self.fc2 = nn.Linear(256, 512)
#         self.bn2 = nn.BatchNorm1d(512)
#         self.fc3 = nn.Linear(512, 256)
#         self.bn3 = nn.BatchNorm1d(256)
        self.fc4 = nn.Linear(dim_features, num_classes)  # CIFAR-10 has 10 classes

    def forward(self, x):
#         x = torch.relu(self.bn1(self.fc1(x)))
#         x = torch.relu(self.bn2(self.fc2(x)))
#         x = torch.relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)  # No activation, CrossEntropyLoss includes softmax
        # Using a linear network for channel network  
        return x

step = 0
def fmcat_loss(RF, RG, P):
    global step
    eps = torch.eye(RF.shape[1], device=RF.device) * 1e-6
    
    lhs = torch.linalg.solve(RF + eps, P + eps)
    rhs = torch.linalg.solve(RG + eps, P.T + eps)
    tsd_m = lhs @ rhs
    tsd = -torch.trace(tsd_m / torch.norm(tsd_m).detach())

    # lhsn = lhs / torch.norm(lhs).detach()
    # rhsn = rhs / torch.norm(rhs).detach()
    # tsd = -torch.trace(lhsn @ rhsn)

    # if step == 0:
    #     # print(RF)
    #     # print(lhs)
    #     print(rhs)
    #     print(torch.linalg.solve(RG, P))
    #     print(RG)
    #     print(P.T)

    step += 1
    # eye = torch.eye(lhs.shape[0]).to(lhs.device)
    # MF = fmri_f.T @ fmri_f
    # MG = smri_f.T @ smri_f
    # reg = torch.square(MF.T @ MF - eye).sum() + torch.square(MG.T @ MG - eye).sum()

    return tsd

def return_cost_trace(RFG, track_cov_estimate_final):
    RF_E = track_cov_estimate_final[:128, :128]
    RG_E = track_cov_estimate_final[128:, 128:]
    P_E = track_cov_estimate_final[:128, 128:]

    RF_EI = torch.inverse(RF_E)
    RG_EI = torch.inverse(RG_E)

    RF = RFG[:128, :128]
    RG = RFG[128:, 128:]
    P = RFG[:128, 128:]

    COST = -RF_EI@RF@RF_EI@P_E@RG_EI@P_E.T \
            + RF_EI@P@RG_EI@P_E.T \
            - RF_EI@P_E@RG_EI@RG@RG_EI@P_E.T \
            + RF_EI@P_E@RG_EI@P.T
    
    TSD = RF_EI@P_E@RG_EI@P_E.T

    return -torch.trace(COST), -torch.trace(TSD)

# for some reasons the adaptive filter is needed
def adaptive_estimation(v_t, beta, square_term, i):
    v_t = beta*v_t + (1-beta)*square_term.detach()
    return v_t, (v_t/(1-beta**i))

def MCA_LOSS_GIVEN_R(RP, track_cov, i, dim):
    cov = RP + torch.eye((RP.shape[0])).cuda()*(.000001)
    track_cov, cov_estimate = adaptive_estimation(track_cov, 0.5, cov, i)

    cov_estimate_f = cov_estimate[:dim, :dim]
    cov_f = cov[:dim, :dim]

    cov_estimate_g = cov_estimate[dim:, dim:]
    cov_g = cov[dim:, dim:]

    LOSS = (torch.linalg.inv(cov_estimate)*cov).sum() - (torch.linalg.inv(cov_estimate_f)*cov_f).sum() -(torch.linalg.inv(cov_estimate_g)*cov_g).sum()
    return track_cov, cov_estimate, LOSS

# torch.cuda.set_device(3)

save_path = './RESULTS_CROSS_SUBJECTS/'
create_folder_if_not_exists(save_path)

NET_1 = Advanced1DCNN_channel(SAMPLE_X.shape[1], 128, 4000).cuda()
NET_2 = Advanced1DCNN_channel(SAMPLE_Y.shape[1], 128, 4000).cuda()

classifier_major = ComplexClassifier(dim_features = 128, num_classes = 3).cuda()
classifier_subtle = ComplexClassifier(dim_features = 128, num_classes = 11).cuda()
classifier_subj = ComplexClassifier(dim_features = 128, num_classes = 25).cuda()
criterion = nn.CrossEntropyLoss()

beta1 = 0.9
beta2 = 0.999

# lr1 = 0.0005
# lr2 = 0.0005
lr1 = 1e-3
lr2 = 1e-3

optimizer_1 = optim.Adam([
    {'params': NET_1.parameters(), 'lr': lr2, 'betas': (beta1, beta2)},
], amsgrad = True)

optimizer_2 = optim.Adam([
    {'params': NET_2.parameters(), 'lr': lr2, 'betas': (beta1, beta2)},
], amsgrad = True)

optimizer_classifier = optim.Adam([
    {'params': classifier_major.parameters(), 'lr': lr2, 'betas': (beta1, beta2)},
    {'params': classifier_subtle.parameters(), 'lr': lr2, 'betas': (beta1, beta2)},
    {'params': classifier_subj.parameters(), 'lr': lr2, 'betas': (beta1, beta2)},
], amsgrad = True)

save_curve = []
eig_list = []
classifier_error = []

test_primary_error_save = []
test_error_save = []

track_cov_final = torch.zeros((128+128)).cuda()
track_cov_estimate_final = torch.zeros((128+128)).cuda()

for i in range(1, 100000):
    batch_size = 12
    batch_indices = torch.randint(0, train_X.shape[0], (batch_size,))
    input_x = train_X[batch_indices]
    input_y = train_Y[batch_indices]
    
    feature_1 = NET_1(input_x.cuda())
    feature_2 = NET_2(input_y.cuda())
    
    RF = (feature_1.T@feature_1)/feature_1.shape[0]
    RG = (feature_2.T@feature_2)/feature_2.shape[0]
    P = (feature_1.T@feature_2)/feature_2.shape[0]

    
    input_dim, output_dim = RF.shape[1], RG.shape[1]
    RFG = torch.zeros((input_dim+output_dim, input_dim+output_dim)).cuda()
    RFG[:input_dim, :input_dim] = RF
    RFG[input_dim:, input_dim:] = RG
    RFG[:input_dim, input_dim:] = P
    RFG[input_dim:, :input_dim] = P.T
    
    # track_cov_final, track_cov_estimate_final, COST = MCA_LOSS_GIVEN_R(RFG, track_cov_final, i, 128)
    # COST, TSD = return_cost_trace(RFG, track_cov_estimate_final)
    loss = fmcat_loss(RF, RF, P)
    label_major_batch, label_subtle_batch, label_subj_batch = train_major_label[batch_indices], train_subtle_label[batch_indices], train_subj_label[batch_indices]

    output_major, output_subtle, output_subj = classifier_major(feature_1.detach()), classifier_subtle(feature_1.detach()), classifier_subj(feature_1.detach())
    # loss_1 = criterion(output_major, label_major_batch.cuda())
    # loss_2 = criterion(output_subtle, label_subtle_batch.cuda())
    # loss_3 = criterion(output_subj, label_subj_batch.cuda())
    # (COST+loss_1+loss_2+loss_3).backward()
    loss.backward()
    # COST.backward()
    # TSD.backward()
    # print("====================================")

    # print("BEFORE UPDATE")
    # plist = list(NET_1.parameters())
    # lp = sum(torch.norm(param.data) for param in plist) / len(plist)
    # # print(f"Param avg norm: {lp}")
    
    # ps = list(NET_1.named_parameters())
    # for name, param in ps[-2:-1]:
    #     if param.grad is not None:
    #         print(name)
    #         print(param.data)

    # print("GRAD")
    # for name, param in ps[-2:-1]:
    #     if param.grad is not None:
    #         print(name)
    #         print(param.grad.data)

    # torch.nn.utils.clip_grad_norm_(NET_1.parameters(), 1.0)
    # torch.nn.utils.clip_grad_norm_(NET_2.parameters(), 1.0)
    optimizer_1.step()  
    optimizer_2.step()  
    optimizer_classifier.step()  

    # print("AFTER UPDATE")
    # plist = list(NET_1.parameters())
    # lp = sum(torch.norm(param.data) for param in plist) / len(plist)
    # print(f"Param avg norm: {lp}")
    
    # ps = list(NET_1.named_parameters())
    # for name, param in ps[-2:-1]:
    #     if param.grad is not None:
    #         print(name)
    #         print(param.data)
    # break
    # print("====================================")
    # for param in NET_2.parameters():
    #     if param.grad is not None:
    #         print(param.grad.data)

#     TSD = torch.logdet(track_cov_estimate_final) - torch.logdet(track_cov_estimate_final[:128, :128]) - torch.logdet(track_cov_estimate_final[128:, 128:])
    # classifier_error.append([TSD.item(), loss_1.item(), loss_2.item(), loss_3.item()])
    optimizer_1.zero_grad()
    optimizer_2.zero_grad()  
    optimizer_classifier.zero_grad()

        
    if i%100 == 0:
        # print('Iteration', i, 'Losses (Trace, Primary Movements, Sub Movements)', TSD.item(), loss_1.item(), loss_2.item())
        # print(f"Step {i}, TSD {TSD.item()}, COST {COST.item()}")
        print(f"Step {i}, loss {loss.item()}")
        # plist = list(NET_1.parameters())
        # lp = sum(torch.norm(param.data) for param in plist) / len(plist)
        # print(f"Param avg norm: {lp}")
                
        # if i%100 == 0:

            # np.save(save_path+'classifier_error_iter{0}.npy'.format(i), classifier_error)
            # torch.save(NET_1.state_dict(), save_path+"NET_1_iter{0}.pth".format(i))
            # torch.save(NET_2.state_dict(), save_path+"NET_2_iter{0}.pth".format(i))
            # torch.save(classifier_major.state_dict(), save_path+"classifier_major_iter{0}.pth".format(i))
            # torch.save(classifier_subtle.state_dict(), save_path+"classifier_subtle_iter{0}.pth".format(i))
            # torch.save(classifier_subj.state_dict(), save_path+"classifier_subj_iter{0}.pth".format(i))

            # NET_1.eval()
            # classifier_major.eval()
            # classifier_subtle.eval()
            # classifier_subj.eval()

            # class_major = []
            # class_minor = []
            # class_subj = []

            # if i%100 == 0:
            #     with torch.no_grad():
            #         for k in range(0, 1000):

            # #         for k in range(0, test_X.shape[0], 1):
            #             output = NET_1(test_X[k:k+1].cuda())
            #             output_major = classifier_major(output)
            #             output_subtle = classifier_subtle(output)
            #             output_subj = classifier_subj(output)

            #             class_major.append(output_major.mean(0).detach().cpu().numpy())
            #             class_minor.append(output_subtle.mean(0).detach().cpu().numpy())
            #             class_subj.append(output_subj.mean(0).detach().cpu().numpy())

            #         major_accuracy = np.mean(np.stack(class_major).argmax(1) == test_major_label[:np.stack(class_major).argmax(1).shape[0]].numpy())
            #         subtle_accuracy = np.mean(np.stack(class_minor).argmax(1) == test_subtle_label[:np.stack(class_minor).argmax(1).shape[0]].numpy())
                    
            #         test_primary_error_save.append(major_accuracy.item())
            #         test_error_save.append(subtle_accuracy.item())

            #         print('Iteration', i, 'Accuracies (Primary Movements, Sub Movements)', np.array(test_primary_error_save).max(), np.array(test_error_save).max())
 
            # NET_1.train()
            # classifier_major.train()
            # classifier_subtle.train()
            # classifier_subj.train()


Folder './RESULTS_CROSS_SUBJECTS/' already exists.
1271956.875 744974.1875
1666906.75 1791732.625
2756906.75 5862288.0
4074855.5 12677123.0
7429062.5 59093812.0
7199569.5 29227276.0
9294102.0 43846408.0
8998592.0 79541280.0
8650842.0 123732208.0
12855560.0 130775544.0
9361105.0 77195336.0
10495752.0 59360344.0
13109358.0 190539696.0
16911172.0 152007344.0
15496298.0 178330032.0
15414418.0 211076048.0
19144766.0 95994752.0
14582872.0 149261616.0
17118486.0 100459144.0
21030280.0 275908640.0
18166410.0 363794560.0
22837336.0 397053664.0
25342320.0 125903768.0
20607782.0 59343480.0
23798346.0 150632096.0
22411536.0 154007408.0
24998378.0 239474384.0
24276828.0 129506416.0
24513048.0 253057344.0
24397806.0 211464432.0
24719596.0 429449952.0
25908254.0 140481232.0
23581842.0 142174592.0
25021272.0 103208944.0
23113356.0 396699744.0
26730130.0 151634368.0
28465012.0 127137496.0
24126472.0 233139712.0
27954452.0 136505328.0
26128172.0 121925176.0
27383120.0 314204064.0
19803442.0 97563968.0
2

KeyboardInterrupt: 